In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dataset_preprocessing import TokenInfo

/home/research/andriv/.conda/envs/pytorch_cuda-11.8/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import pandas as pd

In [4]:
from tqdm import tqdm
import random

In [5]:
token_info = TokenInfo()


...Loading dataset...
...Loading complete...


In [6]:
model_id = "microsoft/phi-1_5"
model_revision = "349cf8b5e81fd5f791d1740da5de1313a0419bbd" # latest as of feb 1st

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
vocab = tokenizer.get_vocab()
len(vocab)

50295

In [8]:
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    revision=model_revision,
    trust_remote_code=True,
    # be careful with this?
    torch_dtype=torch.float16,
    # attn_implementation="flash_attention_2",
)

In [9]:
model

PhiForCausalLM(
  (model): PhiModel(
    (embed_tokens): Embedding(51200, 2048)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-23): 24 x PhiDecoderLayer(
        (self_attn): PhiAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=True)
          (k_proj): Linear(in_features=2048, out_features=2048, bias=True)
          (v_proj): Linear(in_features=2048, out_features=2048, bias=True)
          (dense): Linear(in_features=2048, out_features=2048, bias=True)
          (rotary_emb): PhiRotaryEmbedding()
        )
        (mlp): PhiMLP(
          (activation_fn): NewGELUActivation()
          (fc1): Linear(in_features=2048, out_features=8192, bias=True)
          (fc2): Linear(in_features=8192, out_features=2048, bias=True)
        )
        (input_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (resid_dropout): Dropout(p=0.0, inplace=False)
      )
    )
    (final_layernorm): LayerNorm((2048,), e

In [10]:
top_10k = token_info.top_n(10_000)

In [11]:
token_samples = random.sample(top_10k, 2000)

In [20]:
from importances import compute_delta_loss_importances

In [28]:
importance_dict = {}
# take longer sequences
for i, t_info in tqdm(enumerate(token_samples)):
    examples = token_info.get_prefixes(t_info[0], 50, 10)
    examples = torch.tensor(examples, device="cuda:0")
    importances, sample_inputs = compute_delta_loss_importances(model, examples)
    #print(importances[0])
    #print(sample_input[0])
    importance_dict[t_info] = (importances, sample_inputs)
    if (i+1) % 100 == 0:
        pd.to_pickle(importance_dict, f"importances_inputs/importance_{i}.pkl")        
        del importance_dict
        importance_dict = {}


2000it [31:37,  1.05it/s]


In [ ]:
keys = list(inputs_dict.keys())
inputs_0 = inputs_dict[keys[0]]

In [ ]:
keys

In [ ]:
list(inputs_0.keys())

In [ ]:
inputs_0[0]

In [ ]:
inputs_0[0].size()

In [ ]:
examples.size()